# Task 1.2 Data Analysis Project City Bike NYC

## Goals
This dataset contains a sample of bike trips from the Citi Bike system in New York City.
Each row represents one trip and includes information about the start and end stations, the duration, the
user type, and other contextual data like age, season, temperature, and weekday.
Your goal is to explore this dataset and extract insights through data analysis with pandas.

You'll practice basic pandas operations (loading, exploring, cleaning, transforming, summarizing) and use descriptive statistics and simple visualizations to support your answers.

## 1. Dataset Exploration
- What information does each column contain?
- Are there missing or duplicated values?
- What is the overall time span of the trips?

## 2. Basic Statistics
- What is the average trip duration (in minutes)?
- What is the minimum and maximum duration?
- What are the most common start and end stations?

## 3. Users and Demographics
- How many unique bikes were used?
- What are the proportions of user types (Subscriber vs Customer)?
- What is the age distribution of the users? Which age group uses the service the most?

## 4. Temporal Analysis
- How does the number of trips vary by weekday?
- Which month or season has the most rides?
- What time of day do most trips start?

## 5. Geographic Analysis
- Which station pairs (start → end) appear most often?
- Are there any stations that appear only as start or only as end stations?

## 6. Temperature and Duration
- Is there any visible relationship between temperature and trip duration?
- How does average trip duration vary by season?

## 7. Summary and Interpretation
- Write a short summary (5–10 lines) of your findings.
- Mention patterns, anomalies, or interesting trends you observed.